# APIs externas
Buscaremos:
* Data Set
* JSON
* xml
* csv

preferiblemente abiertos (open data) y actualizados en tiempo real.

Algunos datos, especialmente los privativos, requieren un **Token**.

## Localización de las gasolineras con precio más económico
 * Datos del gobierno de España: https://opendata.esri.es/
  - https://datos.gob.es/en/catalogo/e05024301-precio-de-carburantes-en-las-gasolineras-espanolas
  - https://sedeaplicaciones.minetur.gob.es/ServiciosRESTCarburantes/PreciosCarburantes/help
 * Datos abiertos de la Comunidad de Madrid: https://datos.comunidad.madrid/catalogo/dataset

In [20]:
import requests
import json

peticion=requests.get("https://sedeaplicaciones.minetur.gob.es/ServiciosRESTCarburantes/PreciosCarburantes/EstacionesTerrestres")
contenido = json.loads(peticion.content)
#print(contenido)  # imprime un fichero JSON enorme
print("Información en tiempo real, última actualización", contenido["Fecha"])
listaEstaciones = contenido["ListaEESSPrecio"]
print("Nº estaciones de servicio en España", len(listaEstaciones))

provincia = "Madrid"                # si ponemos España busca en toda España
producto = "Precio Gasoleo A"                       #producto = "Precio Biodiesel"   #producto = "Precio Gasolina 98 E5"
estacionBarata = {}                                 # inicializamos un diccionario. Buscamos la estación de precio menor
for estacion in listaEstaciones:
    if provincia == "España":                       # Se escribe en capitalize
        if estacion[producto] != "":                # no todas las gasolineras tienen Biodiesel, por ejemplo, asi las quitamos
            if estacionBarata == {}:
                estacionBarata = estacion
            else:
                if estacion[producto] < estacionBarata[producto]:
                    estacionBarata = estacion
    elif provincia.upper() == estacion["Provincia"]:      # filtramos por provincia
        if estacion[producto] != "":
            if estacionBarata == {}:
                estacionBarata = estacion
            else:
                if estacion[producto] < estacionBarata[producto]:
                    estacionBarata = estacion

print("Estación de servicio más económica de", provincia,"para",producto)                    
print(estacionBarata["Rótulo"])                          # mostramos el rótulo de la gasolinera
print(estacionBarata["Dirección"], estacionBarata["C.P."], estacionBarata["Localidad"], estacionBarata["Provincia"])
print(estacionBarata[producto], "€/l")

Información en tiempo real, última actualización 30/08/2021 20:02:23
Nº estaciones de servicio en España 11180
Estación de servicio más económica de Madrid para Precio Gasoleo A
BALLENOIL
CALLE SAGITARIO, 2 28023 MADRID MADRID
1,099 €/l


## Desfibriladores de la Comunidad de Madrid

In [19]:
import requests
import json

# Preguntar datos al usuario
print("\nTe voy a pedir tus coordenadas X Y para determinar donde se localiz el desfibrilador mas cercano a tí.")
Ux = float(input("Dime tu coordenada X: ") or 442408)
Uy = float(input("Dime tu coordenada Y: ") or 4479871)

# Se recogen datos de la API de la comunidad de Madrid: https://datos.comunidad.madrid/catalogo/dataset/desfibriladores_externos_fuera_ambito_sanitario/resource/c38446ec-ace1-4d22-942f-5cc4979d19ed
peticion = requests.get(
    "https://datos.comunidad.madrid/catalogo/dataset/35609dd5-9430-4d2e-8198-3eeb277e5282/resource/c38446ec-ace1-4d22-942f-5cc4979d19ed/download/desfibriladores_externos_fuera_ambito_sanitario.json")
contenido = json.loads(peticion.content)
datos = contenido["data"]

# recorremos todos los elementos de datos y calculamos la distancia de cada desfibrilador a nosotros
result = []
for i in range(len(datos)):
    x = ((Ux-float(datos[i]["direccion_coordenada_x"])**.5))
    y = ((Uy-float(datos[i]["direccion_coordenada_y"])**.5))
    result.append((x+y)/2)

# Recogemos la posición del punto más cercano
for j in range(len(result)):
    if (result[j] == min(result, key=abs)):
        posicion = j

# Se imprime el resultado
print("\nEl desfibrilador más cercano a tí se encuentra en",
      datos[posicion]["direccion_via_codigo"], datos[posicion]["direccion_via_nombre"], "en", datos[posicion]["municipio_nombre"])


Te voy a pedir tus coordenadas X Y para determinar donde se localiz el desfibrilador mas cercano a tí.
Dime tu coordenada X: 
Dime tu coordenada Y: 

El desfibrilador más cercano a tí se encuentra en CALLE Hilados , Torrejón de Ardoz
